<div style="background-color:#000;"><img src="pqn.png"></img></div>

These libraries help us analyze financial data and create visualizations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

### Load and prepare our data

We start by loading our data from a CSV file and preparing it for analysis. The file contains many useful columns with Gamma levels. We'll focus on the 0DTE (immediate expiration) support and resistance levels.

In [ ]:
df = pd.read_csv('spy_levels.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
levels = df[["Call Resistance 0DTE", "Put Support 0DTE"]]
levels.columns = ["resistance", "support"]

In [ ]:
spy = yf.download("SPY", start=levels.index.min(), end=levels.index.max()).Close
levels = levels.copy()
levels.loc[:, "spy"] = spy
levels.dropna(inplace=True)

We load our data from a CSV file and convert the 'Date' column to datetime. We set the 'Date' as the index and select specific columns for our analysis. We then download SPY (S&P 500 ETF) closing prices using yfinance and add them to our dataset. Finally, we remove any rows with missing data to ensure our analysis is based on complete information.

### Implement our trading strategy

Next, we define our trading strategy based on support and resistance levels.

In [ ]:
strategy = levels.copy()
strategy["position"] = 0

In [ ]:
strategy.loc[strategy.spy >= strategy.resistance, 'position'] = -1
strategy.loc[strategy.spy <= strategy.support, 'position'] = 1

In [ ]:
strategy.position = (
    strategy
    .position
    .replace(0, np.nan)
    .ffill()
    .fillna(0)
)

In [ ]:
strategy["spy_returns"] = strategy.spy.pct_change()
strategy["strategy_returns"] =  strategy.spy_returns * strategy.position.shift(1)

We create a copy of our data and add a 'position' column. We set our position to -1 (sell) when SPY price is above resistance and 1 (buy) when it's below support. We then forward-fill our positions to maintain them until the next signal. Finally, we calculate the returns for SPY and our strategy by multiplying SPY returns with our previous day's position.

### Evaluate our strategy performance

Finally, we calculate cumulative returns and visualize our strategy's performance.

In [ ]:
strategy['cumulative_returns'] = (1 + strategy.strategy_returns).cumprod()
np.sqrt(252) * strategy.strategy_returns.mean() / strategy.strategy_returns.std()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(strategy.cumulative_returns, label='Strategy')
plt.plot((1 + strategy.spy_returns).cumprod(), label='Buy and Hold')
plt.legend()
plt.title('Strategy Performance vs Buy and Hold')
plt.ylabel('Cumulative Returns')
plt.show()

We calculate the cumulative returns for our strategy and compute its Sharpe ratio. We then create a plot comparing our strategy's performance to a simple buy-and-hold approach. This visualization helps us understand how well our strategy performs compared to passive investing in the S&P 500.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.